Classify CSV Data 

- Calculate LAB conversion from RGB
- Determine Monk Category based on LAB Values

Return CSV File with filled columns

In [4]:
import csv
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_objects import LabColor  
from colormath.color_diff import delta_e_cie1976 
from colormath.color_diff import delta_e_cie1976  
import pandas as pd
import math
from tqdm import tqdm
import numpy as np
from skimage.color import lab2rgb, deltaE_ciede2000


In [11]:
# Define MST reference LAB values
MST_HEX = { 
    1: '#f6ede4',  
    2: '#f3e7db',
    3: '#f7ead0',
    4: '#eadaba',
    5: '#d7bd96',
    6: '#a07e56',
    7: '#825c43',
    8: '#604134',
    9: '#3a312a',
    10: '#292420'
}  

In [12]:
# Define MST reference LAB values
MST_REF = { 
    1: LabColor(lab_l=94.211, lab_a=1.503, lab_b=5.422),  
    2: LabColor(lab_l=92.275, lab_a=2.061, lab_b=7.28), 
    3: LabColor(lab_l=93.091, lab_a=0.216, lab_b=14.205),
    4: LabColor (lab_l=87.573, lab_a=0.459, lab_b=17.748),
    5: LabColor(lab_l=77.902, lab_a=3.471, lab_b=23.136),
    6: LabColor(lab_l=55.142, lab_a=7.783, lab_b=26.74),
    7: LabColor(lab_l=42.47, lab_a=12.325, lab_b=20.53),
    8: LabColor(lab_l=30.678, lab_a=11.667, lab_b=13.335),
    9: LabColor(lab_l=21.069, lab_a=2.69, lab_b=5.964),
    10: LabColor(lab_l=14.61, lab_a=1.482, lab_b=3.525)
}  


In [27]:


def classify_mst_category(L, a, b):
    min_de = float('inf')
    closest_category = 0
    input_color = (L, a, b)  # Store as a tuple instead of LabColor

    for category, ref_lab in MST_REF.items():
        # Extract numerical values from LabColor objects
        ref_values = (ref_lab.lab_l, ref_lab.lab_a, ref_lab.lab_b)
        
        # Calculate deltaE using numerical tuples
        current_de = deltaE_ciede2000(ref_values, input_color)
        
        if current_de < min_de:
            min_de = current_de
            closest_category = category
            
    return closest_category


def process_csv(input_filename, output_filename):
    """
    Reads a CSV file with RGB/HEX values and fills in L*, a*, b*, and monk_category columns.

    Args:
        input_filename: The input CSV file containing RGB/HEX values.
        output_filename: The output CSV file with updated L*, a*, b*, and monk_category columns.
    """
    # Load the CSV file into a DataFrame
    try:
        df = pd.read_csv(input_filename)
        print(f"Successfully loaded {len(df)} rows from {input_filename}.")
     
        # Classify each row into monk_category based on LAB values
        print("Classifying rows into monk_category...")
        df['monk_category'] = df.apply(
            lambda row: classify_mst_category(row['L'], row['a'], row['b']), axis=1)

        # Save the updated DataFrame to a new CSV file
        df.to_csv(output_filename, index=False)
        print(f"Processing complete! Results saved to {output_filename}.")
    
    except FileNotFoundError:
        print(f"Error: File '{input_filename}' not found.")
    
    except Exception as e:
        print(f"An error occurred during processing: {e}")


In [28]:
process_csv("cielab.csv", "output.csv")

Successfully loaded 9846 rows from cielab.csv.
Classifying rows into monk_category...
Processing complete! Results saved to output.csv.
